In [1]:
import numpy as np
import random as rd
import time
import matplotlib.pyplot as plt

In [2]:
def initialize():
    agents = range(n_agents)
    markers = np.random.choice([0, 1], size=(n_agents,), p=[1./2, 1./2])
    prob_vector = np.array([np.array([rd.uniform(0,1),rd.uniform(0,1)]) for _ in range(n_agents)])
    #prob_vector = np.random.uniform(size=n_agents)
    aspiration = np.array([aspiration_value for _ in range(n_agents)])
    return [agents,markers,prob_vector,aspiration]

In [3]:
def select_couple(information):
    if rd.uniform(0,1) > e:
        couple = np.random.choice(information[0],replace=False,size=2)
    else:
        selected_marker = rd.randint(0,1)
        
        same_marker = [i for i,item in enumerate(information[1]) if item == selected_marker]

        couple = np.random.choice(same_marker,size=(2,),replace=False)
    return couple

In [4]:
def interact(information,couple):
    couple_markers = [information[1][ag] for ag in couple[::-1]]
    actions = [int(information[2][ag][couple_markers[i]] < rd.uniform(0,1)) for i,ag in enumerate(couple)]
    delta = reward if all(actions) or not(any(actions)) else 0
    stimulus = [((1+delta+i/2)-information[3][ag])/np.max([abs(information[3][ag] - (1+delta+i/2)) for delta in (0,0.5)])
                for i,ag in enumerate(couple)]
    #Generate Stimulus, update prob_vector
    
    for i,stim in enumerate(stimulus): 
        if stim > 0 :
            if actions[i] == 0:
                information[2][couple[i]][couple_markers[i]] +=(1-information[2][couple[i]][couple_markers[i]])*l*stim
            else:
                information[2][couple[i]][couple_markers[i]] *= (1-l*stim)
        elif stim < 0:
            if actions[i] == 0:
                information[2][couple[i]][couple_markers[i]] += information[2][couple[i]][couple_markers[i]]*l*stim
            else:
                information[2][couple[i]][couple_markers[i]] -= (1-information[2][couple[i]][couple_markers[i]])*l*stim
                
            


In [2]:
3 - np.ones(3)

array([2., 2., 2.])

In [5]:
def save(aspiration_value):
    os.chdir("Animation_"+str(aspiration_value))
    #os.system("ffmpeg -i img%01d.png -framerate 60  -vcodec libx264 -b 800k  -y movie.mp4")
    os.system("ffmpeg -f image2 -i img%01d.jpg -vcodec mpeg4 -b 800k video.avi")
    os.chdir("..")

In [6]:
from IPython.display import clear_output
from matplotlib.ticker import FormatStrFormatter
import os 


def plot_heatmap(prob_vector):
    same_marker = [item[information[1][i]] for i,item in enumerate(information[2])]
    diff_marker = [item[abs(1-information[1][i])] for i,item in enumerate(information[2])]
    corr_coef = np.corrcoef(same_marker,information[1])[0,1]
    hist_2d = np.histogram2d(same_marker,diff_marker,bins=20,range=[[0,1],[0,1]])[0]
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_axes([0,0,1,1])
    ax.set_xlabel(r"$p_{0,\neq}$",fontsize=35)
    ax.set_ylabel(r"$p_{0,=}$",fontsize=35)
    ax.set_yticks(np.arange(0,20,4))
    ax.set_yticklabels(["{:.2f}".format(item) for item in np.arange(1,0,-0.2)],fontsize=18)
    ax.set_xticks(np.arange(0,20,4))
    ax.set_xticklabels(["{:.2f}".format(item) for item in np.arange(1,0,-0.2)],fontsize=18)
    #ax.text(1, 0.5, str("Corr =: {:.2f}".format(corr_coef)),size=35,
    #bbox=dict(boxstyle="round,pad=0.1", fc="white", lw=2), transform=plt.gcf().transFigure)
    #ax.text(1, 0.35, str(r"$\sigma,=$: {:.2f}".format(np.std(same_marker))),size=35,
    #bbox=dict(boxstyle="round,pad=0.1", fc="white", lw=2), transform=plt.gcf().transFigure)
    #ax.text(1, 0.2, str(r"$\sigma,\neq$: {:.2f}".format(np.std(diff_marker))),size=35,
    #bbox=dict(boxstyle="round,pad=0.1", fc="white", lw=2), transform=plt.gcf().transFigure)
    plt.imshow(hist_2d,norm=mynorm,cmap="Reds")
    #plt.show()
    if "Animation_"+str(aspiration_value) not in os.listdir():
        os.mkdir("Animation_"+str(aspiration_value))
    os.chdir("Animation_"+str(aspiration_value))
    plt.savefig("img{}.jpg".format(_//int(n_steps/n_frames)),bbox_inches="tight")
    os.chdir("..")
    plt.close(fig)
    clear_output(wait=True)
    return 

In [10]:
#[prob of choosing 0, aspiration]
#Constant aspiration


e = 0
l = 0.5
n_agents = 500
n_steps = int(n_agents/2*200)
n_frames = 400
aspiration_value = 0.8
reward = 0.5
mynorm = plt.matplotlib.colors.LogNorm(vmin=0, vmax=n_agents,clip=True)

In [8]:
information = initialize()
start = time.perf_counter()
for _ in range(n_steps):
    #Select a random couple
    couple = select_couple(information)
    #A random couple interacts, select decisions
    interact(information,couple)
    #Checking
    if (_%int(n_steps/n_frames) == 0):
        #print(f"{_/n_steps*100:.2f} % reached, elapsed_time: {time.perf_counter()-start:.2f}")
        plot_heatmap(information[2])

In [11]:
save(aspiration_value)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e